# Great Expectations Validation Results Analysis

This notebook reads Great Expectations validation JSON files and presents the data in DataFrames for analysis.

## Overview
- **Purpose**: Analyze validation results from Great Expectations
- **Data Source**: JSON files stored in `BirdiDQ/gx/uncommitted/validations/`
- **Output**: Structured DataFrames for easy analysis and visualization



- **Metadata**: Datasource, data asset, and validation run information
- **Statistics**: Overall success rates and counts
- **Expectations**: Individual expectation results with details
- **Failed Expectations**: Detailed view of failures for remediation
- **Summaries**: Aggregated views by expectation type
- **Exports**: CSV files for further analysis

**Next Steps:**
1. Review failed expectations and investigate root causes
2. Update data quality rules based on findings
3. Re-run validations and compare results over time
4. Integrate with CI/CD pipelines for automated quality checks


In [1]:
import json
import pandas as pd
from pathlib import Path
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options for better DataFrame viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)


## 1. Locate and Load Validation JSON Files


In [2]:
# Define the path to the validations directory
validations_path = Path('/Users/yavin/python_projects/ollama_jupyter/BirdiDQ/gx/uncommitted/validations')

# Find all JSON files recursively
json_files = list(validations_path.rglob('*.json'))

print(f"Found {len(json_files)} validation JSON file(s):\n")
for file in json_files:
    file_size = file.stat().st_size / 1024  # Size in KB
    print(f"  📄 {file.name}")
    print(f"     Path: {file.parent}")
    print(f"     Size: {file_size:.2f} KB\n")


Found 1 validation JSON file(s):

  📄 postgres_sql_nyc_taxi_data-nyc_taxi_data.json
     Path: /Users/yavin/python_projects/ollama_jupyter/BirdiDQ/gx/uncommitted/validations/nyc_taxi_data_onboarding_suite_final/20251005-180117/20251005T180117.575535Z
     Size: 248.59 KB



## 2. Load and Parse Validation Results


In [3]:
def load_validation_json(file_path):
    """Load a validation JSON file and return the parsed data."""
    with open(file_path, 'r') as f:
        return json.load(f)

# Load all validation results
validation_results = []
for json_file in json_files:
    data = load_validation_json(json_file)
    validation_results.append({
        'file_path': str(json_file),
        'file_name': json_file.name,
        'data': data
    })

print(f"✅ Loaded {len(validation_results)} validation result(s)")


✅ Loaded 1 validation result(s)


## 3. Extract Validation Metadata


In [4]:
def extract_metadata(validation_data):
    """Extract high-level metadata from validation results."""
    meta = validation_data.get('meta', {})
    
    return {
        'datasource_name': meta.get('active_batch_definition', {}).get('datasource_name', 'N/A'),
        'data_asset_name': meta.get('active_batch_definition', {}).get('data_asset_name', 'N/A'),
        'expectation_suite_name': meta.get('expectation_suite_name', 'N/A'),
        'run_id': meta.get('run_id', {}).get('run_name', 'N/A') if isinstance(meta.get('run_id'), dict) else str(meta.get('run_id', 'N/A')),
        'validation_time': meta.get('validation_time', 'N/A'),
        'batch_spec_type': meta.get('batch_spec', {}).get('type', 'N/A'),
    }

# Create metadata DataFrame
metadata_list = []
for result in validation_results:
    metadata = extract_metadata(result['data'])
    metadata['file_name'] = result['file_name']
    metadata_list.append(metadata)

metadata_df = pd.DataFrame(metadata_list)
print("\n📊 Validation Metadata:")
metadata_df



📊 Validation Metadata:


,datasource_name,data_asset_name,expectation_suite_name,run_id,validation_time,batch_spec_type,file_name
0,postgres_sql_nyc_taxi_data,nyc_taxi_data,nyc_taxi_data_onboarding_suite_final,20251005-180117,20251005T180117.592126Z,table,postgres_sql_nyc_taxi_data-nyc_taxi_data.json


## 4. Extract Validation Statistics


In [5]:
def extract_statistics(validation_data):
    """Extract validation statistics (success, failure counts)."""
    statistics = validation_data.get('statistics', {})
    
    return {
        'evaluated_expectations': statistics.get('evaluated_expectations', 0),
        'successful_expectations': statistics.get('successful_expectations', 0),
        'unsuccessful_expectations': statistics.get('unsuccessful_expectations', 0),
        'success_percent': statistics.get('success_percent', 0),
    }

# Create statistics DataFrame
statistics_list = []
for result in validation_results:
    stats = extract_statistics(result['data'])
    stats['file_name'] = result['file_name']
    stats['data_asset'] = extract_metadata(result['data'])['data_asset_name']
    statistics_list.append(stats)

statistics_df = pd.DataFrame(statistics_list)
print("\n📈 Validation Statistics:")
statistics_df



📈 Validation Statistics:


,evaluated_expectations,successful_expectations,unsuccessful_expectations,success_percent,file_name,data_asset
0,132,127,5,96.212121,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,nyc_taxi_data


## 5. Extract Individual Expectation Results


In [6]:
def extract_expectation_results(validation_data, file_name):
    """Extract individual expectation results."""
    results = validation_data.get('results', [])
    
    expectations_list = []
    for result in results:
        expectation = result.get('expectation_config', {})
        
        expectations_list.append({
            'file_name': file_name,
            'expectation_type': expectation.get('expectation_type', 'N/A'),
            'column': expectation.get('kwargs', {}).get('column', 'N/A'),
            'success': result.get('success', False),
            'element_count': result.get('result', {}).get('element_count', 'N/A'),
            'unexpected_count': result.get('result', {}).get('unexpected_count', 0),
            'unexpected_percent': result.get('result', {}).get('unexpected_percent', 0),
            'observed_value': result.get('result', {}).get('observed_value', 'N/A'),
        })
    
    return expectations_list

# Create expectations DataFrame
all_expectations = []
for result in validation_results:
    expectations = extract_expectation_results(result['data'], result['file_name'])
    all_expectations.extend(expectations)

expectations_df = pd.DataFrame(all_expectations)
print(f"\n🔍 Individual Expectation Results ({len(expectations_df)} expectations):")
expectations_df.head(20)



🔍 Individual Expectation Results (132 expectations):


,file_name,expectation_type,column,success,element_count,unexpected_count,unexpected_percent,observed_value
0,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_table_row_count_to_be_between,N/A,True,N/A,0,0.0,20000
1,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_table_columns_to_match_set,N/A,True,N/A,0,0.0,"[index, vendor_id, passenger_count, trip_distance, rate_code_id, store_and_fwd_flag, pickup_loca..."
2,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_values_to_not_be_null,index,True,20000,0,0.0,N/A
3,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_min_to_be_between,index,True,N/A,0,0.0,0
4,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_max_to_be_between,index,True,N/A,0,0.0,9999
5,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_values_to_be_between,index,True,20000,0,0.0,N/A
6,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_quantile_values_to_be_between,index,True,N/A,0,0.0,"{'quantiles': [0.25, 0.5, 0.75], 'values': [2499, 4999, 7499]}"
7,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_median_to_be_between,index,True,N/A,0,0.0,4999.5
8,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_mean_to_be_between,index,True,N/A,0,0.0,4999.5
9,postgres_sql_nyc_taxi_data-nyc_taxi_data.json,expect_column_stdev_to_be_between,index,True,N/A,0,0.0,2886.823503


## 6. Analyze Failed Expectations


In [8]:
# Filter for failed expectations
failed_expectations_df = expectations_df[expectations_df['success'] == False].copy()

if len(failed_expectations_df) > 0:
    print(f"\n Failed Expectations ({len(failed_expectations_df)} failures):")
    display(failed_expectations_df[['expectation_type', 'column', 'unexpected_count', 'unexpected_percent', 'observed_value']])
else:
    print("\n✅ All expectations passed! No failures to report.")



 Failed Expectations (5 failures):


,expectation_type,column,unexpected_count,unexpected_percent,observed_value
27,expect_column_mean_to_be_between,trip_distance,0,0.0,2.758991
52,expect_column_mean_to_be_between,fare_amount,0,0.0,12.485031
82,expect_column_mean_to_be_between,tip_amount,0,0.0,1.813903
90,expect_column_mean_to_be_between,tolls_amount,0,0.0,0.293242
112,expect_column_mean_to_be_between,total_amount,0,0.0,15.724231


## 7. Summary Statistics by Expectation Type


In [10]:
# Group by expectation type
expectation_type_summary = expectations_df.groupby('expectation_type').agg({
    'success': ['count', 'sum', 'mean']
}).round(3)

expectation_type_summary.columns = ['Total', 'Passed', 'Success_Rate']
expectation_type_summary['Failed'] = expectation_type_summary['Total'] - expectation_type_summary['Passed']
expectation_type_summary = expectation_type_summary[['Total', 'Passed', 'Failed', 'Success_Rate']]
expectation_type_summary = expectation_type_summary.sort_values('Total', ascending=False)

print("\n Summary by Expectation Type:")
expectation_type_summary



 Summary by Expectation Type:


,Total,Passed,Failed,Success_Rate
expectation_type,,,,
expect_column_max_to_be_between,14,14,0,1.000
expect_column_min_to_be_between,14,14,0,1.000
expect_column_values_to_be_between,14,14,0,1.000
expect_column_values_to_not_be_null,14,14,0,1.000
expect_column_mean_to_be_between,12,7,5,0.583
expect_column_median_to_be_between,12,12,0,1.000
expect_column_quantile_values_to_be_between,12,12,0,1.000
expect_column_stdev_to_be_between,12,12,0,1.000
expect_column_proportion_of_unique_values_to_be_between,8,8,0,1.000


## 8. Export Results to CSV


In [ ]:
# Export DataFrames to CSV
output_dir = Path('/Users/yavin/python_projects/ollama_jupyter/notebooks/great_expectations/exports')
output_dir.mkdir(parents=True, exist_ok=True)

# Export files
metadata_df.to_csv(output_dir / 'validation_metadata.csv', index=False)
statistics_df.to_csv(output_dir / 'validation_statistics.csv', index=False)
expectations_df.to_csv(output_dir / 'expectation_results.csv', index=False)
expectation_type_summary.to_csv(output_dir / 'expectation_type_summary.csv')

if len(failed_expectations_df) > 0:
    failed_expectations_df.to_csv(output_dir / 'failed_expectations.csv', index=False)

print(f"\n✅ Exported results to: {output_dir}")
print("\nFiles created:")
for file in output_dir.glob('*.csv'):
    print(f"  📄 {file.name}")
